## WASABI + MusixMatch API

This file is **not** used for our final analysis as:

1. The free version of the MusixMatch API provides only 30% of the lyrics of each song that is queried
2. There is a daily API limit of 2000 API calls

Instead, we obtain the lyrics from the Genius Song Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# importing Songs dataset
import pandas as pd

dsongs_2 = pd.read_csv('/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/song_lyrics.csv', low_memory=False)
list(dsongs_2.columns)

In [ ]:
# importing Songs dataset
import pandas as pd

dsongs = pd.read_csv('/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/wasabi_songs.csv', sep='\t', low_memory=False)
list(dsongs.columns)

['Unnamed: 0',
 '_id',
 'abstract',
 'albumTitle',
 'album_genre',
 'animux_content',
 'animux_contents',
 'animux_path',
 'animux_paths',
 'arousal',
 'arousal_predicted',
 'artist',
 'artist_lower',
 'availableCountries',
 'award',
 'begin',
 'bpm',
 'chords_metadata',
 'deezer_mapping',
 'disambiguation',
 'end',
 'explicit_content_lyrics',
 'explicit_content_lyrics_predicted',
 'format',
 'gain',
 'genre',
 'has_emotion_tags',
 'has_social_tags',
 'id_album',
 'id_album_deezer',
 'id_artist_deezer',
 'id_song_deezer',
 'id_song_musicbrainz',
 'isClassic',
 'isrc',
 'langdetect_robust',
 'language',
 'language_detect',
 'lastfm_id',
 'length',
 'lengthAlbum',
 'multitrack_file',
 'multitrack_path',
 'newPreview',
 'notes',
 'origin',
 'position',
 'preview',
 'producer',
 'publicationDate',
 'publicationDateAlbum',
 'rank',
 'rdf',
 'recordLabel',
 'recorded',
 'releaseDate',
 'runtime',
 'subject',
 'title',
 'title_accent_fold',
 'title_lower',
 'urlAllmusic',
 'urlAmazon',
 'urlD

In [ ]:
# Selecting only English language songs
dsongs_English = dsongs[dsongs['language_detect'] == 'english']

# Selecting columns we need for songs dataset, we can add other columns or remove current in the future
column_list_for_english_songs = ['artist', 'genre', 'language', 'language_detect', 'title', 'explicit_content_lyrics_predicted']
dsongs_English = dsongs_English[column_list_for_english_songs]
print(len(dsongs_English))

# Saving this file in Datasets folder
dsongs_English.to_csv('/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/wasabi_english_songs.csv')

1314253


In [ ]:
# Imporint artists dataset
import pandas as pd

dartists = pd.read_csv('/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/wasabi_artists.csv', low_memory=False)
list(dartists.columns)

['_id',
 'disambiguation',
 'endArea',
 'endArea.disambiguation',
 'endArea.id',
 'endArea.name',
 'gender',
 'genres',
 'id_artist_musicbrainz',
 'labels',
 'lifeSpan',
 'lifeSpan.begin',
 'lifeSpan.end',
 'lifeSpan.ended',
 'location',
 'location.city',
 'location.country',
 'location.id_city_musicbrainz',
 'locationInfo',
 'members',
 'name',
 'nameVariations_fold',
 'name_accent_fold',
 'rdf',
 'type',
 'urlAllmusic',
 'urlAmazon',
 'urlDiscogs',
 'urlFacebook',
 'urlITunes',
 'urlMusicBrainz',
 'urlMySpace',
 'urlOfficialWebsite',
 'urlPureVolume',
 'urlRateYourMusic',
 'urlSoundCloud',
 'urlSpotify',
 'urlTwitter',
 'urlWikia',
 'urlWikipedia',
 'urlYouTube',
 'id_artist_discogs',
 'abstract',
 'nameVariations',
 'urls',
 'deezerFans',
 'id_artist_deezer',
 'picture',
 'picture.big',
 'picture.medium',
 'picture.small',
 'picture.standard',
 'picture.xl',
 'urlDeezer',
 'urlWikidata',
 'associatedMusicalArtist',
 'dbp_abstract',
 'dbp_genre',
 'recordLabel',
 'subject',
 'members

In [ ]:
# Removing entries where Gender is not Male, Female or Other
dartists_cleaned = dartists.dropna(subset=['gender'])
dartists_cleaned[["gender"]].value_counts()

gender
Male      14309
Female     5881
Other         8
dtype: int64

In [ ]:
# Removing entries where members column in artist is empty []
dartists_cleaned_solo = dartists_cleaned[dartists_cleaned['members'] == '[]']

# Selecting columns we need for artists dataset, we can add other columns or remove current in the future
column_list_for_dartists = ['name', 'gender']
dartists_cleaned_solo = dartists_cleaned_solo[column_list_for_dartists]

In [ ]:
# Checking new artist dataset where gender and solo artists filtered and chosen
dartists_cleaned_solo[["gender"]].value_counts()

gender
Male      8772
Female    4498
Other        6
dtype: int64

In [ ]:
# Choosing artist name from songs dataset with english language and removing the other ones
# (the ones that does not sing in english, because in artist it is not specified fully in what language artist sings, we went forward from Songs to Artists)
idxs = list(dsongs_English.artist.values)

dartists_cleaned_solo = dartists_cleaned_solo[pd.Series(list(dartists_cleaned_solo.name), index=dartists_cleaned_solo.index).isin(idxs)]

In [ ]:
# Checking new artist dataset where english singing artists are kept
dartists_cleaned_solo[["gender"]].value_counts()

gender
Male      6890
Female    3664
Other        6
dtype: int64

In [ ]:
# Saving the file for artists datasrt
dartists_cleaned_solo.to_csv('/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/wasabi_artist_new_dartists.csv')

In [ ]:
df_male = dartists_cleaned_solo[dartists_cleaned_solo['gender'] == 'Male']
df_female = dartists_cleaned_solo[dartists_cleaned_solo['gender'] == 'Female']

# Determine the smaller size among the two groups
min_size = min(len(df_male), len(df_female))

# Truncate both datasets to have the same size
df_male_balanced = df_male.sample(min_size)
df_female_balanced = df_female.sample(min_size)

# Combine the balanced datasets
df_balanced = pd.concat([df_male_balanced, df_female_balanced])
df_balanced.to_csv('/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/wasabi_new_dartists_balanced.csv')

In [ ]:
df_balanced[["gender"]].value_counts()

gender
Female    3664
Male      3664
dtype: int64

In [ ]:
print(len(dsongs_English))

1314253


In [ ]:
# Again removing songs which does not belong to artists in wasabi_new_dartists_balanced.csv file
idxs2 = list(df_balanced.name.values)

dsongs2_English = dsongs_English[pd.Series(list(dsongs_English.artist), index=dsongs_English.index).isin(idxs2)]

In [ ]:
print(len(dsongs2_English))

176583


In [ ]:
dsongs2_English.to_csv('/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/wasabi_new_songs_english_balanced_artists.csv')

In [ ]:
import requests

class MusixMatch:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.musixmatch.com/ws/1.1/"

    def _make_request(self, method, params):
        params['apikey'] = self.api_key
        response = requests.get(self.base_url + method, params=params)
        response.raise_for_status()
        return response.json()

    def search_track(self, track_name):
        method = 'track.search'
        params = {'q_track': track_name, 'page_size': 1, 'page': 1, 's_track_rating': 'desc'}
        data = self._make_request(method, params)
        track_list = data['message']['body']['track_list']
        return track_list[0]['track']['track_id'] if track_list else None

    def get_lyrics(self, track_id):
        method = 'track.lyrics.get'
        params = {'track_id': track_id}
        data = self._make_request(method, params)
        return data['message']['body']['lyrics']['lyrics_body'] if data['message']['body'] else None

api_key = 'da10a106601943f9899e0695231a139d'
musixmatch = MusixMatch(api_key)

# Search for a track and get its lyrics
track_name = "Shape of You"
track_id = musixmatch.search_track(track_name)
if track_id:
    lyrics = musixmatch.get_lyrics(track_id)
    print("Lyrics:\n", lyrics)
else:
    print("Track not found.")

# get the lyrics

Lyrics:
 The club isn't the best place to find a lover
So the bar is where I go (mm-mm)
Me and my friends at the table doing shots
Drinking fast, and then we talk slow (mm-mm)
You come over and start up a conversation with just me
And trust me, I'll give it a chance now (mm-mm)
Take my hand, stop, put "Van the Man" on the jukebox
And then we start to dance
And now I'm singing like

Girl, you know I want your love
Your love was handmade for somebody like me
Come on now, follow my lead
I may be crazy, don't mind me
Say, "Boy, let's not talk too much
Grab on my waist and put that body on me"
Come on now, follow my lead
Come, come on now, follow my lead
(Mm-mm)

I'm in love with the shape of you
We push and pull like a magnet do
Although my heart is falling too
I'm in love with your body
Last night you were in my room
And now my bed sheets smell like you
Every day discovering something brand new
Oh, I'm in love with your body

Oh I, oh I, oh I, oh I
Oh, I'm in love with your body
Oh I, oh 

In [ ]:
def append_lyrics_to_csv(csv_file_path, api_key):
    # Initialize MusixMatch
    musixmatch = MusixMatch(api_key)

    # Load CSV file
    df = pd.read_csv(csv_file_path)

    def get_lyrics_for_track(track_name):
        track_id = musixmatch.search_track(track_name)
        if track_id:
            return musixmatch.get_lyrics(track_id)
        else:
            return "Lyrics Not Found"

    # Fetch lyrics for each track and append as a new column
    df['lyrics'] = df['title'].apply(get_lyrics_for_track)

    # Save the updated DataFrame back to CSV
    df.to_csv('updated_' + csv_file_path)

# Example usage
csv_file_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/wasabi_new_songs_english_balanced_artists.csv'
api_key = 'da10a106601943f9899e0695231a139d'  # Your Musixmatch API key
append_lyrics_to_csv(csv_file_path, api_key)

NameError: ignored

In [ ]:
import pandas as pd

def append_lyrics_to_csv(csv_file_path, api_key, start_row=0):
    musixmatch = MusixMatch(api_key)
    df = pd.read_csv(csv_file_path)

    # Add a 'lyrics_addbyus' column if it doesn't exist
    if 'lyrics_addbyus' not in df.columns:
        df['lyrics_addbyus'] = pd.NA

    for i, row in df.iterrows():
        if i < start_row:
            continue  # Skip rows until the starting row is reached

        # Check if lyrics are already present
        if pd.isna(df.at[i, 'lyrics_addbyus']):
            print(i)
            track_name = row['title']  # Replace 'title' with your actual column name if different
            track_id = musixmatch.search_track(track_name)
            if track_id:
                lyrics = musixmatch.get_lyrics(track_id)
                df.at[i, 'lyrics_addbyus'] = lyrics  # Add lyrics to the DataFrame
            else:
                df.at[i, 'lyrics_addbyus'] = "Lyrics Not Found"

            # Save progress after each update directly to the same CSV file
            df.to_csv(csv_file_path, index=False)

# Example usage
csv_file_path = '/content/drive/MyDrive/Praktikum - NLP Applications/Datasets/wasabi_new_songs_english_balanced_artists.csv'
api_key = 'da10a106601943f9899e0695231a139d'  # Your Musixmatch API key
start_row = 14810  # Set the row number from which you want to start fetching lyrics, change it everytime you ran the code
append_lyrics_to_csv(csv_file_path, api_key, start_row)

<ipython-input-4-320b7c5fc5d0>:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


14810
14811
14812
14813
14814
14815
14816
14817
14818
14819
14820
14821
14822
14823
14824
14825
14826
14827
14828
14829
14830
14831
14832
14833
14834
14835
14836
14837
14838
14839
14840
14841
14842
14843
14844
14845
14846
14847
14848
14849
14850
14851
14852
14853
14854
14855
14856
14857
14858
14859
14860
14861
14862
14863
14864
14865
14866
14867
14868
14869
14870
14871
14872
14873
14874
14875
14876
14877
14878
14879
14880
14881
14882
14883
14884
14885
14886
14887
14888
14889
14890
14891
14892
14893
14894
14895
14896
14897
14898
14899
14900
14901
14902
14903
14904
14905
14906
14907
14908
14909
14910
14911
14912
14913
14914
14915
14916
14917
14918
14919
14920
14921
14922
14923
14924
14925
14926
14927
14928
14929
14930
14931
14932
14933
14934
14935
14936
14937
14938
14939
14940
14941
14942
14943
14944
14945
14946
14947
14948
14949
14950
14951
14952
14953
14954
14955
14956
14957
14958
14959
14960
14961
14962
14963
14964
14965
14966
14967
14968
14969
14970
14971
14972
14973
14974
14975
1497

TypeError: ignored